In [ ]:
!pip install mysql-connector-python

In [ ]:
import mysql.connector
from tabulate import tabulate

# Function to execute SQL queries and display results
def execute_query(cursor, query):
    cursor.execute(query)
    columns = cursor.description
    results = cursor.fetchall()
    if results:
        headers = [column[0] for column in columns]
        data = [headers] + [list(row) for row in results]
        print("\nResults:")
        print(tabulate(data, headers="firstrow", tablefmt="grid"))
    else:
        print("\nNo results found.")

# Connect to MySQL database
try:
    connection = mysql.connector.connect(
        user='root',
        password='',
        host='localhost',
        database=''
    )
    if connection.is_connected():
        print("Connected to MySQL database.")

        cursor = connection.cursor()

        # Prompt user for input and execute corresponding query
        while True:
            print("\nWhich query would you like to execute?")
            print("1. Total deliveries and success rate for a specific date with delivery agent")
            print("2. Total products returned by customers with payments over a specific amount")
            print("3. Customers with more than particular number of orders and their total orders")
            print("4. Total orders by membership type with validity period over specific year period")
            print("5. Exit")

            choice = input("Enter your choice (1-5): ")

            if choice == "1":
                delivery_date = input("Enter delivery date (YYYY-MM-DD): ")
                query = f"SELECT d.DeliveryAgentID, COUNT(*) AS TotalDeliveries, SUM(CASE WHEN d.DeliverySuccess = 1 THEN 1 ELSE 0 END) AS SuccessfulDeliveries, AVG(CASE WHEN d.DeliverySuccess = 1 THEN 1 ELSE 0 END) AS SuccessRate FROM Delivery AS d WHERE d.DeliveryDate = '{delivery_date}' GROUP BY d.DeliveryAgentID;"
                execute_query(cursor, query)

            elif choice == "2":
                amount = input("Enter a specific amount:")
                query = f"SELECT c.CustomerID, c.CustomerName, COUNT(rp.ReturnProductID) AS TotalProductsReturned FROM Customers AS c JOIN PaymentProfiles AS pp ON c.CustomerID = pp.CustomerID JOIN Payment AS p ON pp.PaymentProfileID = p.PaymentProfileID JOIN OrderDetails AS od ON p.OrderNum = od.OrderNum JOIN ReturnedProduct AS rp ON od.OrderNum = rp.OrderNum WHERE p.Amount > {amount} GROUP BY c.CustomerID, c.CustomerName;"
                execute_query(cursor, query)

            elif choice == "3":
                total_orders = input("Enter the amount of orders:")
                query = f"SELECT c.CustomerID, c.CustomerName, COALESCE(COUNT(od.OrderNum), 0) AS TotalOrders FROM OrderDetails AS od RIGHT JOIN Payment AS p ON od.OrderNum = p.OrderNum JOIN PaymentProfiles AS pp ON p.PaymentProfileID = pp.PaymentProfileID RIGHT JOIN Customers AS c ON pp.CustomerID = c.CustomerID GROUP BY c.CustomerID, c.CustomerName HAVING TotalOrders > {total_orders};"
                execute_query(cursor, query)

            elif choice == "4":
                validity_period = input("Enter the validity period:")
                query = f"SELECT m.MembershipName, COUNT(*) AS TotalOrders FROM OrderDetails AS od JOIN Payment AS p ON od.OrderNum = p.OrderNum JOIN PaymentProfiles AS pp ON p.PaymentProfileID = pp.PaymentProfileID JOIN Customers AS c ON pp.CustomerID = c.CustomerID JOIN MembershipType AS m ON c.MembershipTypeID = m.MembershipTypeID WHERE c.MembershipTypeID IN (SELECT MembershipTypeID FROM MembershipType WHERE ValidityPeriod > {validity_period}) GROUP BY m.MembershipName;"
                execute_query(cursor, query)

            elif choice == "5":
                break

            else:
                print("Invalid choice. Please enter a number from 1 to 5.")

except mysql.connector.Error as e:
    print("Error connecting to MySQL database:", e)

finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
        print("\nMySQL connection closed.")


In [ ]:
import mysql.connector
from tkinter import *
from tkinter import messagebox
from tabulate import tabulate

# Function to execute SQL queries and display results
def execute_query():
    query_choice = query_var.get()
    if query_choice == "1":
        delivery_date = input_date.get()
        if not delivery_date:
            messagebox.showwarning("Missing Input", "Please enter delivery date.")
            return
        query = f"SELECT d.DeliveryAgentID, COUNT(*) AS TotalDeliveries, SUM(CASE WHEN d.DeliverySuccess = 1 THEN 1 ELSE 0 END) AS SuccessfulDeliveries, AVG(CASE WHEN d.DeliverySuccess = 1 THEN 1 ELSE 0 END) AS SuccessRate FROM Delivery AS d WHERE d.DeliveryDate = '{delivery_date}' GROUP BY d.DeliveryAgentID;"
    elif query_choice == "2":
        amount = input_amount.get()
        if not amount:
            messagebox.showwarning("Missing Input", "Please enter a specific amount.")
            return
        query = f"SELECT c.CustomerID, c.CustomerName, COUNT(rp.ReturnProductID) AS TotalProductsReturned FROM Customers AS c JOIN PaymentProfiles AS pp ON c.CustomerID = pp.CustomerID JOIN Payment AS p ON pp.PaymentProfileID = p.PaymentProfileID JOIN OrderDetails AS od ON p.OrderNum = od.OrderNum JOIN ReturnedProduct AS rp ON od.OrderNum = rp.OrderNum WHERE p.Amount > {amount} GROUP BY c.CustomerID, c.CustomerName;"
    elif query_choice == "3":
        total_orders = input_orders.get()
        if not total_orders:
            messagebox.showwarning("Missing Input", "Please enter the amount of orders.")
            return
        query = f"SELECT c.CustomerID, c.CustomerName, COALESCE(COUNT(od.OrderNum), 0) AS TotalOrders FROM OrderDetails AS od RIGHT JOIN Payment AS p ON od.OrderNum = p.OrderNum JOIN PaymentProfiles AS pp ON p.PaymentProfileID = pp.PaymentProfileID RIGHT JOIN Customers AS c ON pp.CustomerID = c.CustomerID GROUP BY c.CustomerID, c.CustomerName HAVING TotalOrders > {total_orders};"
    elif query_choice == "4":
        validity_period = input_validity.get()
        if not validity_period:
            messagebox.showwarning("Missing Input", "Please enter the validity period.")
            return
        query = f"SELECT m.MembershipName, COUNT(*) AS TotalOrders FROM OrderDetails AS od JOIN Payment AS p ON od.OrderNum = p.OrderNum JOIN PaymentProfiles AS pp ON p.PaymentProfileID = pp.PaymentProfileID JOIN Customers AS c ON pp.CustomerID = c.CustomerID JOIN MembershipType AS m ON c.MembershipTypeID = m.MembershipTypeID WHERE c.MembershipTypeID IN (SELECT MembershipTypeID FROM MembershipType WHERE ValidityPeriod > {validity_period}) GROUP BY m.MembershipName;"

    try:
        cursor.execute(query)
        columns = cursor.description
        results = cursor.fetchall()
        if results:
            headers = [column[0] for column in columns]
            data = [headers] + [list(row) for row in results]
            result_text.config(state=NORMAL)
            result_text.delete("1.0", END)
            result_text.insert(END, tabulate(data, headers="firstrow", tablefmt="grid"))
            result_text.config(state=DISABLED)
        else:
            messagebox.showinfo("No results", "No results found.")
    except mysql.connector.Error as e:
        messagebox.showerror("Error", f"Error executing query:\n{e}")

# Function to update input messages and query description based on the selected query
def update_input_messages(*args):
    query_choice = query_var.get()
    query_desc_label.config(text=query_descriptions.get(query_choice, ""))
    if query_choice == "1":
        input_date_label.pack(anchor=W)
        input_date.pack(anchor=W)
        input_date.focus_set()  # Set focus to input_date field
    else:
        input_date_label.pack_forget()
        input_date.pack_forget()
    if query_choice == "2":
        input_amount_label.pack(anchor=W)
        input_amount.pack(anchor=W)
        input_amount.focus_set()  # Set focus to input_amount field
    else:
        input_amount_label.pack_forget()
        input_amount.pack_forget()
    if query_choice == "3":
        input_orders_label.pack(anchor=W)
        input_orders.pack(anchor=W)
        input_orders.focus_set()  # Set focus to input_orders field
    else:
        input_orders_label.pack_forget()
        input_orders.pack_forget()
    if query_choice == "4":
        input_validity_label.pack(anchor=W)
        input_validity.pack(anchor=W)
        input_validity.focus_set()  # Set focus to input_validity field
    else:
        input_validity_label.pack_forget()
        input_validity.pack_forget()
    
    # Clear the output window
    result_text.config(state=NORMAL)
    result_text.delete("1.0", END)
    result_text.config(state=DISABLED)
    
    execute_button.focus_set()  # Set focus to execute button

    # Set focus to the appropriate input field
    if query_choice == "1":
        input_date.focus_set()
    elif query_choice == "2":
        input_amount.focus_set()
    elif query_choice == "3":
        input_orders.focus_set()
    elif query_choice == "4":
        input_validity.focus_set()
    
# Bind the function to the OptionMenu selection event
query_var.trace_add('write', update_input_messages)
    
    

# Connect to MySQL database
try:
    connection = mysql.connector.connect(
        user='root',
        password='',
        host='localhost',
        database=''
    )
    if connection.is_connected():
        print("Connected to MySQL database.")
        cursor = connection.cursor()

        # Create GUI window
        root = Tk()
        root.title("MySQL Query Executor")
        
        # Message label
        message_label = Label(root, text="Please select a query number from the below dropdown:")
        message_label.pack(anchor=W)

        # Query descriptions
        query_descriptions = {
            "1": "Total deliveries and success rate for a specific date with delivery agent",
            "2": "Total products returned by customers with payments over a specific amount",
            "3": "Customers with more than particular number of orders and their total orders",
            "4": "Total orders by membership type with validity period over specific year period"
        }

        # Dropdown menu for selecting query
        query_var = StringVar()
        query_var.set("1")  # Set default value
        query_var.trace("w", update_input_messages)
        query_menu = OptionMenu(root, query_var, *query_descriptions.keys())
        query_menu.pack(anchor=W)

        # Query description label
        query_desc_label = Label(root, text=query_descriptions["1"])
        query_desc_label.pack(anchor=W)

        # Input fields
        input_date_label = Label(root, text="Enter delivery date (YYYY-MM-DD):")
        input_date = Entry(root)
        input_amount_label = Label(root, text="Enter a specific amount:")
        input_amount = Entry(root)
        input_orders_label = Label(root, text="Enter the amount of orders:")
        input_orders = Entry(root)
        input_validity_label = Label(root, text="Enter the validity period:")
        input_validity = Entry(root)
        
        # Pack input fields
        input_date_label.pack(anchor=W, padx=10, pady=5)
        input_date.pack(anchor=W, padx=10, pady=5)
        input_amount_label.pack(anchor=W, padx=10, pady=5)
        input_amount.pack(anchor=W, padx=10, pady=5)
        input_orders_label.pack(anchor=W, padx=10, pady=5)
        input_orders.pack(anchor=W, padx=10, pady=5)
        input_validity_label.pack(anchor=W, padx=10, pady=5)
        input_validity.pack(anchor=W, padx=10, pady=5)
        

        # Text area to display query results
        result_text = Text(root, height=20, width=100)
        result_text.pack()
        
        # Execute Query button
        execute_button = Button(root, text="Execute Query", command=execute_query)
        execute_button.pack(anchor=W)

        # Set focus to the first input field
        input_date.focus_set()


        update_input_messages()  # Call to initially show input box for query 1
        root.mainloop()

except mysql.connector.Error as e:
    messagebox.showerror("Error", f"Error connecting to MySQL database:\n{e}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("\nMySQL connection closed.")


In [2]:
import mysql.connector
from tkinter import *
from tkinter import messagebox
from tabulate import tabulate

# Function to execute SQL queries and display results
def execute_query():
    query_choice = query_var.get()
    if query_choice == "1":
        delivery_date = input_date.get()
        if not delivery_date:
            messagebox.showwarning("Missing Input", "Please enter delivery date.")
            return
        query = f"SELECT d.DeliveryAgentID, COUNT(*) AS TotalDeliveries, SUM(CASE WHEN d.DeliverySuccess = 1 THEN 1 ELSE 0 END) AS SuccessfulDeliveries, AVG(CASE WHEN d.DeliverySuccess = 1 THEN 1 ELSE 0 END) AS SuccessRate FROM Delivery AS d WHERE d.DeliveryDate = '{delivery_date}' GROUP BY d.DeliveryAgentID;"
    elif query_choice == "2":
        amount = input_amount.get()
        if not amount:
            messagebox.showwarning("Missing Input", "Please enter a specific amount.")
            return
        query = f"SELECT c.CustomerID, c.CustomerName, COUNT(rp.ReturnProductID) AS TotalProductsReturned FROM Customers AS c JOIN PaymentProfiles AS pp ON c.CustomerID = pp.CustomerID JOIN Payment AS p ON pp.PaymentProfileID = p.PaymentProfileID JOIN OrderDetails AS od ON p.OrderNum = od.OrderNum JOIN ReturnedProduct AS rp ON od.OrderNum = rp.OrderNum WHERE p.Amount > {amount} GROUP BY c.CustomerID, c.CustomerName;"
    elif query_choice == "3":
        total_orders = input_orders.get()
        if not total_orders:
            messagebox.showwarning("Missing Input", "Please enter the amount of orders.")
            return
        query = f"SELECT c.CustomerID, c.CustomerName, COALESCE(COUNT(od.OrderNum), 0) AS TotalOrders FROM OrderDetails AS od RIGHT JOIN Payment AS p ON od.OrderNum = p.OrderNum JOIN PaymentProfiles AS pp ON p.PaymentProfileID = pp.PaymentProfileID RIGHT JOIN Customers AS c ON pp.CustomerID = c.CustomerID GROUP BY c.CustomerID, c.CustomerName HAVING TotalOrders > {total_orders};"
    elif query_choice == "4":
        validity_period = input_validity.get()
        if not validity_period:
            messagebox.showwarning("Missing Input", "Please enter the validity period.")
            return
        query = f"SELECT m.MembershipName, COUNT(*) AS TotalOrders FROM OrderDetails AS od JOIN Payment AS p ON od.OrderNum = p.OrderNum JOIN PaymentProfiles AS pp ON p.PaymentProfileID = pp.PaymentProfileID JOIN Customers AS c ON pp.CustomerID = c.CustomerID JOIN MembershipType AS m ON c.MembershipTypeID = m.MembershipTypeID WHERE c.MembershipTypeID IN (SELECT MembershipTypeID FROM MembershipType WHERE ValidityPeriod > {validity_period}) GROUP BY m.MembershipName;"
    elif query_choice == "5":
        customer_ID = input_customer_ID.get()
        if not customer_ID:
            messagebox.showwarning("Missing Input", "Please enter the Customer ID.")
            return
        query = f"SELECT c.CustomerID, c.CustomerName,od.ProductID, p.ProductName, od.ProductQuantity FROM OrderDetails od JOIN Product p ON od.ProductID = p.ProductID JOIN ( SELECT ps.OrderNum, c.CustomerName, c.CustomerID FROM Payment ps JOIN PaymentProfiles pp ON ps.PaymentProfileID = pp.PaymentProfileID  JOIN Customers c ON pp.CustomerID = c.CustomerID WHERE c.CustomerID = {customer_ID} ) AS c ON od.OrderNum = c.OrderNum;"
    elif query_choice == "6":
        delivery_status = input_delivery_status.get()
        if not delivery_status:
            messagebox.showwarning("Missing Input", "Please enter the Delivery Status as 1 or 0.")
            return
        query = f"SELECT c.CustomerID, c.CustomerName,od.ProductID, p.ProductName,od.ProductQuantity, d.DeliverySuccess, d.DeliveryDate FROM OrderDetails od INNER JOIN Payment ps ON od.OrderNum = ps.OrderNum INNER JOIN Delivery d ON ps.OrderNum = d.OrderNum INNER JOIN Product p ON od.ProductID = p.ProductID JOIN Customers c ON d.CustomerID = c.CustomerID WHERE d.DeliverySuccess = {delivery_status} Order by 3 ASC LIMIT 30;"

    try:
        cursor.execute(query)
        columns = cursor.description
        results = cursor.fetchall()
        if results:
            headers = [column[0] for column in columns]
            data = [headers] + [list(row) for row in results]
            result_text.config(state=NORMAL)
            result_text.delete("1.0", END)
            result_text.insert(END, tabulate(data, headers="firstrow", tablefmt="grid"))
            result_text.config(state=DISABLED)
        else:
            messagebox.showinfo("No results", "No results found.")
    except mysql.connector.Error as e:
        messagebox.showerror("Error", f"Error executing query:\n{e}")

# Function to update input messages and query description based on the selected query
def update_input_messages(*args):
    query_choice = query_var.get()
    query_desc_label.config(text=query_descriptions.get(query_choice, ""))
    if query_choice == "1":
        input_date_label.pack(anchor=W)
        input_date.pack(anchor=W)
        input_date.focus_set()  # Set focus to input_date field
        input_amount_label.pack_forget()
        input_amount.pack_forget()
        input_orders_label.pack_forget()
        input_orders.pack_forget()
        input_validity_label.pack_forget()
        input_validity.pack_forget()
        input_customer_ID_label.pack_forget()
        input_customer_ID.pack_forget()
        input_delivery_status_label.pack_forget()
        input_delivery_status.pack_forget()
    elif query_choice == "2":
        input_amount_label.pack(anchor=W)
        input_amount.pack(anchor=W)
        input_amount.focus_set()  # Set focus to input_amount field
        input_date_label.pack_forget()
        input_date.pack_forget()
        input_orders_label.pack_forget()
        input_orders.pack_forget()
        input_validity_label.pack_forget()
        input_validity.pack_forget()
        input_customer_ID_label.pack_forget()
        input_customer_ID.pack_forget()
        input_delivery_status_label.pack_forget()
        input_delivery_status.pack_forget()
    elif query_choice == "3":
        input_orders_label.pack(anchor=W)
        input_orders.pack(anchor=W)
        input_orders.focus_set()  # Set focus to input_orders field
        input_date_label.pack_forget()
        input_date.pack_forget()
        input_amount_label.pack_forget()
        input_amount.pack_forget()
        input_validity_label.pack_forget()
        input_validity.pack_forget()
        input_customer_ID_label.pack_forget()
        input_customer_ID.pack_forget()
        input_delivery_status_label.pack_forget()
        input_delivery_status.pack_forget()
    elif query_choice == "4":
        input_validity_label.pack(anchor=W)
        input_validity.pack(anchor=W)
        input_validity.focus_set()  # Set focus to input_validity field
        input_date_label.pack_forget()
        input_date.pack_forget()
        input_amount_label.pack_forget()
        input_amount.pack_forget()
        input_orders_label.pack_forget()
        input_orders.pack_forget()
        input_customer_ID_label.pack_forget()
        input_customer_ID.pack_forget()
        input_delivery_status_label.pack_forget()
        input_delivery_status.pack_forget()
    elif query_choice == "5":
        input_customer_ID_label.pack(anchor=W)
        input_customer_ID.pack(anchor=W)
        input_customer_ID.focus_set()  # Set focus to input_customer_ID field
        input_date_label.pack_forget()
        input_date.pack_forget()
        input_amount_label.pack_forget()
        input_amount.pack_forget()
        input_orders_label.pack_forget()
        input_orders.pack_forget()
        input_validity_label.pack_forget()
        input_validity.pack_forget()
        input_delivery_status_label.pack_forget()
        input_delivery_status.pack_forget()
    elif query_choice == "6":
        input_delivery_status_label.pack(anchor=W)
        input_delivery_status.pack(anchor=W)
        input_delivery_status.focus_set()  # Set focus to input_delivery_status field
        input_date_label.pack_forget()
        input_date.pack_forget()
        input_amount_label.pack_forget()
        input_amount.pack_forget()
        input_orders_label.pack_forget()
        input_orders.pack_forget()
        input_validity_label.pack_forget()
        input_validity.pack_forget()
        input_customer_ID_label.pack_forget()
        input_customer_ID.pack_forget()
        
    execute_button.focus_set()  # Set focus to execute button


# Connect to MySQL database
try:
    connection = mysql.connector.connect(
        user='root',
        password='',
        host='localhost',
        database=''
    )
    if connection.is_connected():
        print("Connected to MySQL database.")
        cursor = connection.cursor()

        # Create GUI window
        root = Tk()
        root.title("MySQL Query Executor")
        
         # Message label
        message_label = Label(root, text="Please select a query number from the below dropdown:")
        message_label.pack(anchor=W)

        # Query descriptions
        query_descriptions = {
            "1": "Query Description: Total deliveries and success rate for a specific date with delivery agent",
            "2": "Query Description: Total products returned by customers with payments over a specific amount",
            "3": "Query Description: Customers with more than particular number of orders and their total orders",
            "4": "Query Description: Total orders by membership type with validity period over specific year period",
            "5": "Query Description: To Retrieve Product Details for Orders Placed by CustomerID",
            "6": "Query Description: To Retrieve top 30 products Ordered by Customers Who Received Deliveries Successfull or Unsuccesfull ordered by Product ID"
        }

        # Dropdown menu for selecting query
        query_var = StringVar()
        query_var.set("1")  # Set default value
        query_var.trace("w", update_input_messages)
        query_menu = OptionMenu(root, query_var, *query_descriptions.keys())
        query_menu.pack(anchor=W)

        # Query description label
        query_desc_label = Label(root, text=query_descriptions["1"])
        query_desc_label.pack(anchor=W)

        # Input fields
        input_date_label = Label(root, text="Enter delivery date (YYYY-MM-DD):")
        input_date = Entry(root)
        

       
        
        input_amount_label = Label(root, text="Enter a specific amount:")
        input_amount = Entry(root)
        input_orders_label = Label(root, text="Enter the amount of orders:")
        input_orders = Entry(root)
        input_validity_label = Label(root, text="Enter the validity period:")
        input_validity = Entry(root)
        input_customer_ID_label = Label(root, text="Enter the CustomerID:")
        input_customer_ID = Entry(root)
        input_delivery_status_label = Label(root, text="Enter the Delivery Status as 1 or 0:")
        input_delivery_status = Entry(root)
      
        
        # Text area to display query results
        result_text = Text(root, height=50, width=150)
        result_text.pack()
        
        # Execute Query button
        execute_button = Button(root, text="Execute Query", command=execute_query)
        execute_button.pack(anchor=W)

        update_input_messages()  # Call to initially show input box for query 1
        root.mainloop()

except mysql.connector.Error as e:
    messagebox.showerror("Error", f"Error connecting to MySQL database:\n{e}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("\nMySQL connection closed.")


Connected to MySQL database.

MySQL connection closed.
